Reads assignment for AF546 data. only counts are collected.    
-- note: used the same segmentation mask as sequencing images

In [10]:
import sys, os
import numpy as np
import pandas as pd
import seaborn as sns
from starmap.sequencing import *
from scipy.io import loadmat
from tqdm import tqdm
import tifffile
from skimage.measure import regionprops

helper

In [11]:
def load_af546(fpath, reads_file):
    S = loadmat(os.path.join(fpath, reads_file))
    points = S['merged_points']
    temp = np.zeros(points.shape)
    temp[:, 0] = np.round(points[:, 1]-1)
    temp[:, 1] = np.round(points[:, 0]-1)
    temp[:, 2] = np.round(points[:, 2]-1)
    print(f"Number of reads: {len(points)}")
    
    return temp

In [12]:
# IO
ppath = 'Z:/Data/Processed/2022-09-30-Rena-Cardiomyocyte64Gene/'
opath = os.path.join(ppath, 'output', '2022-10-15-merged-points')
area_list = [i+1 for i in range(6)]

In [13]:
structure_dict = {'whole_cell': 'cell',
                 'nucleus': 'nuclei'}

In [16]:
for area in area_list:
    area_out_path = os.path.join(opath, f'Area_{area}')
    area_result = []
    for current_structure in structure_dict:
        # Load segmentation
        seg_path = os.path.join(ppath, 'output', '2022-10-17-3D-segmentation')
        current_seg = tifffile.imread(os.path.join(seg_path, structure_dict[current_structure], f'Area{area}_{structure_dict[current_structure]}_3Dseg.tif'))
        points = load_af546(os.path.join(opath, f'Area_{area}'), "merged_AF546_points.mat")
        points = points.astype(int)
        # Assign reads to current structure
        reads_assignment = current_seg[points[:, 2], points[:, 0], points[:, 1]]
        total_cells = len(np.unique(current_seg)) - 1
        cell_ct = [0] * total_cells
        for i in range(total_cells):
            n_reads = np.count_nonzero(reads_assignment == i+1)
            cell_ct[i] += n_reads
        area_result.append(cell_ct)
    area_result = pd.DataFrame(np.array(area_result).T, columns = ['whole_cell', 'nucleus'])
    area_result.to_csv(os.path.join(area_out_path, 'AF546_cell_counts.csv'), index=False)

Number of reads: 5247
Number of reads: 5247
Number of reads: 6727
Number of reads: 6727
Number of reads: 7240
Number of reads: 7240
Number of reads: 4471
Number of reads: 4471
Number of reads: 6281
Number of reads: 6281
Number of reads: 6088
Number of reads: 6088
